In [1]:
import os
import sys
sys.path.append(os.path.abspath(".."))
import pandas as pd
import torch
from preprocessing import *
import time
from PC_transformer import PCTransformer

# set_seed(43)

In [2]:
df = pd.read_csv(r'..\output_csv\ANL-Intrepid-2009-1.swf.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68936 entries, 0 to 68935
Data columns (total 18 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   job_id                    68936 non-null  float64
 1   submit_time               68936 non-null  float64
 2   wait_time                 68936 non-null  float64
 3   run_time                  68936 non-null  float64
 4   num_allocated_processors  68936 non-null  float64
 5   avg_cpu_time_used         68936 non-null  float64
 6   used_memory               68936 non-null  float64
 7   requested_processors      68936 non-null  float64
 8   requested_time            68936 non-null  float64
 9   requested_memory          68936 non-null  float64
 10  status                    68936 non-null  float64
 11  user_id                   68936 non-null  float64
 12  group_id                  68936 non-null  float64
 13  executable_id             68936 non-null  float64
 14  queue_

In [3]:
df.head()

,job_id,submit_time,wait_time,run_time,num_allocated_processors,avg_cpu_time_used,used_memory,requested_processors,requested_time,requested_memory,status,user_id,group_id,executable_id,queue_id,partition_id,preceding_job_id,think_time
0,1.0,0.0,6680.0,7560.0,2048.0,-1.0,-1.0,2048.0,10800.0,-1.0,-1.0,1.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0
1,2.0,7.0,14297.0,7568.0,2048.0,-1.0,-1.0,2048.0,10800.0,-1.0,-1.0,1.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0
2,3.0,1590.0,17322.0,7561.0,2048.0,-1.0,-1.0,2048.0,10800.0,-1.0,-1.0,1.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0
3,4.0,2205.0,61.0,14972.0,8192.0,-1.0,-1.0,8192.0,21600.0,-1.0,-1.0,2.0,-1.0,-1.0,2.0,-1.0,-1.0,-1.0
4,5.0,2566.0,17357.0,7571.0,2048.0,-1.0,-1.0,2048.0,10800.0,-1.0,-1.0,1.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0


In [4]:
feature_columns = ['requested_processors', 'requested_time', 'submit_time', 'wait_time', 'user_id', 'queue_id']
target_column = 'run_time'

# Hyperparameters
input_dim = len(feature_columns)    # Example input dimension
d_model = 64      # Model dimension
num_heads = 8     # Number of attention heads
d_ff = 256        # Feed-forward dimension
num_layers = 3    # Number of transformer blocks
output_dim = 1    # Output dimension
batch_size = 128  # Batch size
seq_len = 20      # Length of data group
num_epochs = 20   # Number of training epochs
dropout = 0.2    # Dropout rate
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

dataloaders, scaler = prepare_data_seq(df, feature_columns, target_column, statuss=-1, seq_len=seq_len, batch_size=batch_size)

In [5]:
rmse_lst, mae_lst, mse_lst, r2_lst, infer_time_lst = [], [], [], [], []

for iter in range(10):
    print(f'\nIteration {iter + 1} / 100')
    # Create model
    model = PCTransformer(input_dim, d_model, num_heads, d_ff, num_layers, output_dim, dropout).to(device)

    # Train the model
    model.train_model(model, dataloaders, num_epochs, device, scaler)

    model.load_state_dict(torch.load('../models_HCMUT/best_pc_transformer_model.pth', map_location=device))
    rmse, mae, mse, r2, infer_time = model.evaluate_model(model, dataloaders['test'], device, scaler, input_dim)

    rmse_lst.append(rmse)
    mae_lst.append(mae)
    mse_lst.append(mse)
    r2_lst.append(r2)
    infer_time_lst.append(infer_time)

    print(f'\nFinal Test Metrics after loading best model:')
    print(f'Test RMSE: {rmse:.4f}, Test MAE: {mae:.4f}, Test MSE: {mse:.4f}, Test R2: {r2:.4f}')
    print(f'Inference Time: {infer_time} seconds per sample')

# Save results to CSV
results_df = pd.DataFrame({
    'RMSE': rmse_lst,
    'MAE': mae_lst,
    'MSE': mse_lst,
    'R2': r2_lst,
    'Inference Time': infer_time_lst
})
results_df.to_csv('../output_ANL/pc_transformer_results.csv', index=False)
# Final evaluation on test set
# test_rmse, test_mae, test_mse, test_r2 = model.evaluate_model(model, dataloaders['test'], device, scaler, input_dim)
# print(f'\nFinal Test Metrics:')
# print(f'Test RMSE: {test_rmse:.4f}, Test MAE: {test_mae:.4f}, Test MSE: {test_mse:.4f}, Test R2: {test_r2:.4f}')
# print(f'Training time: {end - start:.2f} seconds')


Iteration 1 / 100
Epoch [1/20]
Train Loss: 0.2499, Val Loss: 0.2351
Val RMSE: 10677.5626, Val MAE: 4838.8142, Val MSE: 114010343.2287, Val R2: 0.0277
Epoch [2/20]
Train Loss: 0.2047, Val Loss: 0.1666
Val RMSE: 8776.3371, Val MAE: 3786.5024, Val MSE: 77024092.4598, Val R2: 0.3431
Epoch [3/20]
Train Loss: 0.1261, Val Loss: 0.1202
Val RMSE: 8121.4767, Val MAE: 2913.1390, Val MSE: 65958383.9998, Val R2: 0.4375
Epoch [4/20]
Train Loss: 0.1117, Val Loss: 0.1196
Val RMSE: 8125.1593, Val MAE: 3028.3703, Val MSE: 66018214.4195, Val R2: 0.4370
Epoch [5/20]
Train Loss: 0.1066, Val Loss: 0.1224
Val RMSE: 8249.2308, Val MAE: 2922.9273, Val MSE: 68049808.7015, Val R2: 0.4197
Epoch [6/20]
Train Loss: 0.1038, Val Loss: 0.1125
Val RMSE: 8014.2697, Val MAE: 2682.6615, Val MSE: 64228519.0554, Val R2: 0.4522
Epoch [7/20]
Train Loss: 0.1013, Val Loss: 0.1119
Val RMSE: 8015.8666, Val MAE: 2653.6085, Val MSE: 64254116.8525, Val R2: 0.4520
Epoch [8/20]
Train Loss: 0.0998, Val Loss: 0.1118
Val RMSE: 8096.3005